In [1]:
import cv2
import numpy as np
import math
print(cv2.__version__)

4.6.0


In [3]:
def find_aruco_markers(img):
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)
    parameters = cv2.aruco.DetectorParameters_create()
    corners, ids, _ = cv2.aruco.detectMarkers(img, aruco_dict, parameters=parameters)
    return corners, ids

def calculate_robot_position(corners, ids):
    top_left_marker = corners[np.where(ids == 1)[0][0]]
    bottom_right_marker = corners[np.where(ids == 2)[0][0]]
    robot_marker = corners[np.where(ids == 153)[0][0]]

    top_left_center = np.mean(top_left_marker, axis=1)[0]
    bottom_right_center = np.mean(bottom_right_marker, axis=1)[0]
    robot_center = np.mean(robot_marker, axis=1)[0]

    width = bottom_right_center[0] - top_left_center[0]
    height = bottom_right_center[1] - top_left_center[1]

    x = robot_center[0] - top_left_center[0]
    y = robot_center[1] - top_left_center[1]

    x_norm = x / width
    y_norm = y / height

    robot_vector = robot_marker[0][1] - robot_marker[0][0]
    theta = math.atan2(robot_vector[1], robot_vector[0])

    return (top_left_center, bottom_right_center, (x_norm, y_norm, theta))

def locate_robot(img):
    corners, ids = find_aruco_markers(img)
    if ids is None or len(ids) < 3:
        return None, None, None
    return calculate_robot_position(corners, ids)

def draw_robot_marker(img, robot_position, top_left, bottom_right):
    if robot_position is None:
        return img

    x_norm, y_norm, theta = robot_position
    top_left_x, top_left_y = top_left
    bottom_right_x, bottom_right_y = bottom_right

    width = bottom_right_x - top_left_x
    height = bottom_right_y - top_left_y
    x_img = int(top_left_x + x_norm * width)
    y_img = int(top_left_y + y_norm * height)

    arrow_length = 50
    end_x = int(x_img + arrow_length * math.cos(theta))
    end_y = int(y_img + arrow_length * math.sin(theta))

    cv2.circle(img, (x_img, y_img), 5, (0, 0, 255), -1)
    cv2.arrowedLine(img, (x_img, y_img), (end_x, end_y), (0, 0, 255), 2, tipLength=0.3)

    cv2.putText(img, f'Robot: ({x_img}, {y_img})', (x_img + 10, y_img), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    return img

def draw_bounding_box(img, top_left, bottom_right):
    if top_left is None or bottom_right is None:
        return img

    top_left_x, top_left_y = int(top_left[0]), int(top_left[1])
    bottom_right_x, bottom_right_y = int(bottom_right[0]), int(bottom_right[1])
    cv2.rectangle(img, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 0), 2)

    cv2.putText(img, f'Top Left: ({top_left_x}, {top_left_y})', (top_left_x + 10, top_left_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(img, f'Bottom Right: ({bottom_right_x}, {bottom_right_y})', (bottom_right_x + 10, bottom_right_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return img

def process_camera_feed():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open video capture")
        return

    cv2.namedWindow("Camera Feed with ArUco Markers", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Camera Feed with ArUco Markers", 800, 600)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame")
            break

        corners, ids = find_aruco_markers(frame)
        top_left, bottom_right, robot_position = locate_robot(frame)

        if ids is None or len(ids) < 3:
            frame_with_text = frame.copy()
            cv2.putText(frame_with_text, '< 3 ArUco markers detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            frame_with_text = frame
            frame_with_arrow = draw_robot_marker(frame_with_text, robot_position, top_left, bottom_right)
            frame_with_bounding_box = draw_bounding_box(frame_with_arrow, top_left, bottom_right)

        cv2.imshow('Camera Feed with ArUco Markers', frame_with_text)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    process_camera_feed()


KeyboardInterrupt: 